In [1]:
import os 
from sqlalchemy import create_engine
import pandas as pd

In [2]:
user1 = os.environ['PAYMEE_SONTATA_DB_USER']
password = os.environ['PAYMEE_SONTATA_DB_PASSWORD']
db = "Sonata_Connect"
server = "172.17.130.216"   

engine = create_engine(f"mssql+pyodbc://{user1}:{password}@{server}/{db}?driver=ODBC+Driver+17+for+SQL+Server")


In [3]:
pd.set_option('display.max_columns', 500)

In [4]:
sql_query = f"""SET NOCOUNT ON;SELECT task_id, task_description, status, priority, assigned_to, created_by, completion_date, created_on, deadline_date, category, review, feedback FROM [{db}].[dbo].accounts_task_details ORDER BY created_on;SET NOCOUNT OFF"""
all_tasks_df=pd.read_sql_query(sql_query,engine)


sql_query = f"""SET NOCOUNT ON;EXEC [{db}].[dbo].[SP_UserHierarchy_Dynamic_07Jan23] @userid = 6892;SET NOCOUNT OFF"""
df=pd.read_sql_query(sql_query,engine)


In [5]:
all_tasks_df.head()

,task_id,task_description,status,priority,assigned_to,created_by,completion_date,created_on,deadline_date,category,review,feedback
0,129116,Customer (Customer id: Sdl:Bhr:01:006) is inte...,Pending,1,29938,2271,None,2023-04-01,2023-04-01,7,None,None
1,129117,Customer (Customer id: Rab:cit:01:236:037) has...,Pending,1,30482,27199,None,2023-04-01,2023-04-01,10,None,None
2,129118,Customer (Customer id: Bab:cit:01:189:043) has...,Pending,1,27245,27199,None,2023-04-01,2023-04-01,10,None,None
3,129119,Customer (Customer id: 1397268) will pay an am...,Pending,1,27815,27815,None,2023-04-01,2023-04-01,3,None,None
4,129120,Customer (Customer id: Lko:mhn:01:116:004) has...,Pending,1,31063,27199,None,2023-04-01,2023-04-01,10,None,None


In [6]:
df

,BUId,BUType,ReportingBUId,buname,UserID,UserName,U_BUID,RoleId,RoleName
0,1,1,0,Head Office,6892,BRIJESH KUMAR VISHWAKARMA,1,41,IT DO
1,650,2,1,Prayagraj Division,7540,ANSHUMAN TIWARI,650,39,Disbursement Confirmer
2,650,2,1,Prayagraj Division,9739,LAKSHMAN SINGH,650,34,DIVISION HEAD
3,650,2,1,Prayagraj Division,9739,LAKSHMAN SINGH,650,44,Credit Committee
4,650,2,1,Prayagraj Division,15183,ABDUL KADIR ANSARI,650,58,Audit
...,...,...,...,...,...,...,...,...,...
2997,2325,5,2104,Nirmali,27196,Mithilesh Kumar Paswan,2325,55,BRO
2998,2326,5,2206,Chandwa,27375,Rahul Raj,2326,13,Branch Manager
2999,2326,5,2206,Chandwa,31817,MODASSHIR RAZA,2326,55,BRO
3000,2327,5,607,Kurha,8699,SHASHIDHAR PASWAN,2327,13,Branch Manager


In [7]:
reqdf = pd.merge(all_tasks_df, df, how='left', left_on='assigned_to', right_on='UserID').drop_duplicates(subset=['task_id'])
reqdf.head()

,task_id,task_description,status,priority,assigned_to,created_by,completion_date,created_on,deadline_date,category,review,feedback,BUId,BUType,ReportingBUId,buname,UserID,UserName,U_BUID,RoleId,RoleName
0,129116,Customer (Customer id: Sdl:Bhr:01:006) is inte...,Pending,1,29938,2271,None,2023-04-01,2023-04-01,7,None,None,459.0,5.0,2141.0,Burhar,29938.0,Virendra Singh gound,459.0,55.0,BRO
1,129117,Customer (Customer id: Rab:cit:01:236:037) has...,Pending,1,30482,27199,None,2023-04-01,2023-04-01,10,None,None,79.0,5.0,584.0,Raibareli,30482.0,DHARMENDRA BAHADUR,79.0,55.0,BRO
2,129118,Customer (Customer id: Bab:cit:01:189:043) has...,Pending,1,27245,27199,None,2023-04-01,2023-04-01,10,None,None,8.0,5.0,583.0,Barabanki city,27245.0,ROLI KUMARI,8.0,55.0,BRO
3,129119,Customer (Customer id: 1397268) will pay an am...,Pending,1,27815,27815,None,2023-04-01,2023-04-01,3,None,None,424.0,5.0,2139.0,Guna,27815.0,AAKASH YADAV,424.0,55.0,BRO
4,129120,Customer (Customer id: Lko:mhn:01:116:004) has...,Pending,1,31063,27199,None,2023-04-01,2023-04-01,10,None,None,184.0,5.0,583.0,Mohanlalganj,31063.0,SATYAM SINGH,184.0,55.0,BRO


In [8]:
len(reqdf)

28850

In [9]:
div_df = df[(df['RoleId']==34)&(df['RoleName']=='DIVISION HEAD')][['U_BUID','UserID','UserName','buname']]
div_df.rename(columns={'U_BUID':'Div_BUID','buname':'DivName','UserID':'Div_UserID','UserName':'Div_UserName'}, inplace=True)
reg_df = df[(df['RoleId']==35)&(df['RoleName']=='REGION HEAD')][['U_BUID','UserID','UserName','buname','ReportingBUId']]
reg_df.rename(columns={'U_BUID':'Region_BUID','ReportingBUId':'Region_ReportingBUId','buname':'RegionName','UserID':'Region_UserID','UserName':'Region_UserName'}, inplace=True)
hub_df = df[(df['RoleId']==36)&(df['RoleName']=='HUB HEAD')][['U_BUID','UserID','UserName','buname','ReportingBUId']]
hub_df.rename(columns={'U_BUID':'Hub_BUID','ReportingBUId':'Hub_ReportingBUId','buname':'HubName','UserID':'Hub_UserID','UserName':'Hub_UserName'}, inplace=True)
bm_df = df[(df['RoleId']==13)&(df['RoleName']=='Branch Manager')][['U_BUID','buname','UserID','UserName','ReportingBUId']]
bm_df.rename(columns={'U_BUID':'BM_BUID','ReportingBUId':'BM_ReportingBUId','buname':'BranchName','UserID':'BM_UserID','UserName':'BM_UserName'}, inplace=True)
bro_df = df[(df['RoleId']==55)&(df['RoleName']=='BRO')][['U_BUID','UserID','UserName']]
bro_df.rename(columns={'U_BUID':'BRO_BUID','UserID':'BRO_UserID','UserName':'BRO_UserName'}, inplace=True)
M1 = pd.merge(bro_df,bm_df,left_on='BRO_BUID',right_on='BM_BUID',how='left').drop_duplicates(subset='BRO_UserID')
M2 = pd.merge(M1,hub_df,left_on='BM_ReportingBUId',right_on='Hub_BUID',how='left').drop_duplicates(subset='BRO_UserID')
M3 = pd.merge(M2,reg_df,left_on='Hub_ReportingBUId',right_on='Region_BUID',how='left').drop_duplicates(subset='BRO_UserID')
dF = pd.merge(M3,div_df,left_on='Region_ReportingBUId',right_on='Div_BUID',how='left').drop_duplicates(subset='BRO_UserID')

In [10]:
dF.head()

,BRO_BUID,BRO_UserID,BRO_UserName,BM_BUID,BranchName,BM_UserID,BM_UserName,BM_ReportingBUId,Hub_BUID,Hub_UserID,Hub_UserName,HubName,Hub_ReportingBUId,Region_BUID,Region_UserID,Region_UserName,RegionName,Region_ReportingBUId,Div_BUID,Div_UserID,Div_UserName,DivName
0,591,4829,ANKIT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,3353,RAHUL KUMAR,2.0,Sirathu,4797.0,PINTU KUMAR YADAV,545.0,545.0,1692.0,Shivlal pal,BHARWARI,2180.0,2180.0,18044.0,KAMLESH YADAV,Kaushambi,650.0,650.0,9739.0,LAKSHMAN SINGH,Prayagraj Division
2,2,25715,DHEERAJ KUMAR,2.0,Sirathu,4797.0,PINTU KUMAR YADAV,545.0,545.0,1692.0,Shivlal pal,BHARWARI,2180.0,2180.0,18044.0,KAMLESH YADAV,Kaushambi,650.0,650.0,9739.0,LAKSHMAN SINGH,Prayagraj Division
3,2,27274,Saurabh Kushwaha,2.0,Sirathu,4797.0,PINTU KUMAR YADAV,545.0,545.0,1692.0,Shivlal pal,BHARWARI,2180.0,2180.0,18044.0,KAMLESH YADAV,Kaushambi,650.0,650.0,9739.0,LAKSHMAN SINGH,Prayagraj Division
4,2,31073,NEERAJ KUMAR,2.0,Sirathu,4797.0,PINTU KUMAR YADAV,545.0,545.0,1692.0,Shivlal pal,BHARWARI,2180.0,2180.0,18044.0,KAMLESH YADAV,Kaushambi,650.0,650.0,9739.0,LAKSHMAN SINGH,Prayagraj Division


In [13]:
reqdf.sample()

,task_id,task_description,status,priority,assigned_to,created_by,completion_date,created_on,deadline_date,category,review,feedback,BUId,BUType,ReportingBUId,buname,UserID,UserName,U_BUID,RoleId,RoleName
15649,159549,Customer (Customer id: Bha:Ala:411:135) has co...,Pending,1,31689,9731,None,2023-04-05,2023-04-05,10,None,,411.0,5.0,573.0,Bandikui,31689.0,VISHVENDRA SINGH,411.0,55.0,BRO


In [49]:
reqdf = pd.merge(reqdf, dF, how='left', left_on='UserID', right_on = 'BRO_UserID' )

In [50]:
reqdf.head()

,task_id,task_description,status,priority,assigned_to,created_by,completion_date,created_on,deadline_date,category,review,feedback,BUId,BUType,ReportingBUId,buname,UserID,UserName,U_BUID,RoleId,RoleName,BRO_BUID_x,BRO_UserID_x,BRO_UserName_x,BM_BUID_x,BranchName_x,BM_UserID_x,BM_UserName_x,BM_ReportingBUId_x,Hub_BUID_x,Hub_UserID_x,Hub_UserName_x,HubName_x,Hub_ReportingBUId_x,Region_BUID_x,Region_UserID_x,Region_UserName_x,RegionName_x,Region_ReportingBUId_x,Div_BUID_x,Div_UserID_x,Div_UserName_x,DivName_x,BRO_BUID_y,BRO_UserID_y,BRO_UserName_y,BM_BUID_y,BranchName_y,BM_UserID_y,BM_UserName_y,BM_ReportingBUId_y,Hub_BUID_y,Hub_UserID_y,Hub_UserName_y,HubName_y,Hub_ReportingBUId_y,Region_BUID_y,Region_UserID_y,Region_UserName_y,RegionName_y,Region_ReportingBUId_y,Div_BUID_y,Div_UserID_y,Div_UserName_y,DivName_y
0,129116,Customer (Customer id: Sdl:Bhr:01:006) is inte...,Pending,1,29938,2271,None,2023-04-01,2023-04-01,7,None,None,459.0,5.0,2141.0,Burhar,29938.0,Virendra Singh gound,459.0,55.0,BRO,459.0,29938.0,Virendra Singh gound,459.0,Burhar,3366.0,rajesh Sahu,2141.0,2141.0,18022.0,Deepak Jagati,Burhar,2183.0,2183.0,1622.0,Pradeep kachhi,Katni,652.0,652.0,15190.0,JITENDRA YADAV,Jabalpur,459.0,29938.0,Virendra Singh gound,459.0,Burhar,3366.0,rajesh Sahu,2141.0,2141.0,18022.0,Deepak Jagati,Burhar,2183.0,2183.0,1622.0,Pradeep kachhi,Katni,652.0,652.0,15190.0,JITENDRA YADAV,Jabalpur
1,129117,Customer (Customer id: Rab:cit:01:236:037) has...,Pending,1,30482,27199,None,2023-04-01,2023-04-01,10,None,None,79.0,5.0,584.0,Raibareli,30482.0,DHARMENDRA BAHADUR,79.0,55.0,BRO,79.0,30482.0,DHARMENDRA BAHADUR,79.0,Raibareli,1884.0,Vinod Kumar,584.0,584.0,15051.0,DHARMENDRA PRATAP SINGH,Raibarely,1002.0,1002.0,0.0,REGION HEAD NOT POSTED,Lucknow,651.0,651.0,7535.0,RAKESH KUMAR CHAUDHARY,Lucknow Division,79.0,30482.0,DHARMENDRA BAHADUR,79.0,Raibareli,1884.0,Vinod Kumar,584.0,584.0,15051.0,DHARMENDRA PRATAP SINGH,Raibarely,1002.0,1002.0,0.0,REGION HEAD NOT POSTED,Lucknow,651.0,651.0,7535.0,RAKESH KUMAR CHAUDHARY,Lucknow Division
2,129118,Customer (Customer id: Bab:cit:01:189:043) has...,Pending,1,27245,27199,None,2023-04-01,2023-04-01,10,None,None,8.0,5.0,583.0,Barabanki city,27245.0,ROLI KUMARI,8.0,55.0,BRO,8.0,27245.0,ROLI KUMARI,8.0,Barabanki city,3682.0,BHUPENDRA SINGH,583.0,583.0,3245.0,Krishan Kumar Yadav,Barabanki,1002.0,1002.0,0.0,REGION HEAD NOT POSTED,Lucknow,651.0,651.0,7535.0,RAKESH KUMAR CHAUDHARY,Lucknow Division,8.0,27245.0,ROLI KUMARI,8.0,Barabanki city,3682.0,BHUPENDRA SINGH,583.0,583.0,3245.0,Krishan Kumar Yadav,Barabanki,1002.0,1002.0,0.0,REGION HEAD NOT POSTED,Lucknow,651.0,651.0,7535.0,RAKESH KUMAR CHAUDHARY,Lucknow Division
3,129119,Customer (Customer id: 1397268) will pay an am...,Pending,1,27815,27815,None,2023-04-01,2023-04-01,3,None,None,424.0,5.0,2139.0,Guna,27815.0,AAKASH YADAV,424.0,55.0,BRO,424.0,27815.0,AAKASH YADAV,424.0,Guna,2702.0,Prahlad Ahirwar,2139.0,2139.0,259.0,Vikash Namdev,GUNA,1013.0,1013.0,16347.0,Susheel Kumar Yadav,Gwalior,2156.0,2156.0,10851.0,Vinod singh,Gwalior,424.0,27815.0,AAKASH YADAV,424.0,Guna,2702.0,Prahlad Ahirwar,2139.0,2139.0,259.0,Vikash Namdev,GUNA,1013.0,1013.0,16347.0,Susheel Kumar Yadav,Gwalior,2156.0,2156.0,10851.0,Vinod singh,Gwalior
4,129120,Customer (Customer id: Lko:mhn:01:116:004) has...,Pending,1,31063,27199,None,2023-04-01,2023-04-01,10,None,None,184.0,5.0,583.0,Mohanlalganj,31063.0,SATYAM SINGH,184.0,55.0,BRO,184.0,31063.0,SATYAM SINGH,184.0,Mohanlalganj,21.0,MOHD IRFAN,583.0,583.0,3245.0,Krishan Kumar Yadav,Barabanki,1002.0,1002.0,0.0,REGION HEAD NOT POSTED,Lucknow,651.0,651.0,7535.0,RAKESH KUMAR CHAUDHARY,Lucknow Division,184.0,31063.0,SATYAM SINGH,184.0,Mohanlalganj,21.0,MOHD IRFAN,583.0,583.0,3245.0,Krishan Kumar Yadav,Barabanki,1002.0,1002.0,0.0,REGION HEAD NOT POSTED,Lucknow,651.0,651.0,7535.0,RAKESH KUMAR CHAUDHARY,Lucknow Division


In [15]:
len(dF)

2090

In [16]:
div_df

,Div_BUID,Div_UserID,Div_UserName,DivName
2,650,9739,LAKSHMAN SINGH,Prayagraj Division
14,651,7535,RAKESH KUMAR CHAUDHARY,Lucknow Division
20,652,15190,JITENDRA YADAV,Jabalpur
26,653,10849,KESHRAJ YADAV,Muzaffarpur
34,654,7662,Sandeep Singh,Moradabad
51,2154,17756,Alok Singh,Varanasi
58,2155,7575,CHANDRA SHEKHAR,Jaipur
66,2156,10851,Vinod singh,Gwalior
70,2157,10846,SHASHI BHUSHAN SINGH,Saharanpur
76,2158,15111,Ajay Kumar Singh,Patna


In [17]:
reqdf = pd.merge(reqdf, dF, how='left', left_on='UserID', right_on = 'BRO_UserID' )

In [18]:
reqdf.head()

,task_id,task_description,status,priority,assigned_to,created_by,completion_date,created_on,deadline_date,category,review,feedback,BUId,BUType,ReportingBUId,buname,UserID,UserName,U_BUID,RoleId,RoleName,BRO_BUID,BRO_UserID,BRO_UserName,BM_BUID,BranchName,BM_UserID,BM_UserName,BM_ReportingBUId,Hub_BUID,Hub_UserID,Hub_UserName,HubName,Hub_ReportingBUId,Region_BUID,Region_UserID,Region_UserName,RegionName,Region_ReportingBUId,Div_BUID,Div_UserID,Div_UserName,DivName
0,129116,Customer (Customer id: Sdl:Bhr:01:006) is inte...,Pending,1,29938,2271,None,2023-04-01,2023-04-01,7,None,None,459.0,5.0,2141.0,Burhar,29938.0,Virendra Singh gound,459.0,55.0,BRO,459.0,29938.0,Virendra Singh gound,459.0,Burhar,3366.0,rajesh Sahu,2141.0,2141.0,18022.0,Deepak Jagati,Burhar,2183.0,2183.0,1622.0,Pradeep kachhi,Katni,652.0,652.0,15190.0,JITENDRA YADAV,Jabalpur
1,129117,Customer (Customer id: Rab:cit:01:236:037) has...,Pending,1,30482,27199,None,2023-04-01,2023-04-01,10,None,None,79.0,5.0,584.0,Raibareli,30482.0,DHARMENDRA BAHADUR,79.0,55.0,BRO,79.0,30482.0,DHARMENDRA BAHADUR,79.0,Raibareli,1884.0,Vinod Kumar,584.0,584.0,15051.0,DHARMENDRA PRATAP SINGH,Raibarely,1002.0,1002.0,0.0,REGION HEAD NOT POSTED,Lucknow,651.0,651.0,7535.0,RAKESH KUMAR CHAUDHARY,Lucknow Division
2,129118,Customer (Customer id: Bab:cit:01:189:043) has...,Pending,1,27245,27199,None,2023-04-01,2023-04-01,10,None,None,8.0,5.0,583.0,Barabanki city,27245.0,ROLI KUMARI,8.0,55.0,BRO,8.0,27245.0,ROLI KUMARI,8.0,Barabanki city,3682.0,BHUPENDRA SINGH,583.0,583.0,3245.0,Krishan Kumar Yadav,Barabanki,1002.0,1002.0,0.0,REGION HEAD NOT POSTED,Lucknow,651.0,651.0,7535.0,RAKESH KUMAR CHAUDHARY,Lucknow Division
3,129119,Customer (Customer id: 1397268) will pay an am...,Pending,1,27815,27815,None,2023-04-01,2023-04-01,3,None,None,424.0,5.0,2139.0,Guna,27815.0,AAKASH YADAV,424.0,55.0,BRO,424.0,27815.0,AAKASH YADAV,424.0,Guna,2702.0,Prahlad Ahirwar,2139.0,2139.0,259.0,Vikash Namdev,GUNA,1013.0,1013.0,16347.0,Susheel Kumar Yadav,Gwalior,2156.0,2156.0,10851.0,Vinod singh,Gwalior
4,129120,Customer (Customer id: Lko:mhn:01:116:004) has...,Pending,1,31063,27199,None,2023-04-01,2023-04-01,10,None,None,184.0,5.0,583.0,Mohanlalganj,31063.0,SATYAM SINGH,184.0,55.0,BRO,184.0,31063.0,SATYAM SINGH,184.0,Mohanlalganj,21.0,MOHD IRFAN,583.0,583.0,3245.0,Krishan Kumar Yadav,Barabanki,1002.0,1002.0,0.0,REGION HEAD NOT POSTED,Lucknow,651.0,651.0,7535.0,RAKESH KUMAR CHAUDHARY,Lucknow Division


In [19]:
cols_req = [
    'created_on',
    'DivName',
    'RegionName',
    'HubName',
    'BranchName',
    'assigned_to',
    'created_by',
    'priority',
    'task_id',
    'category',
    'task_description',
    'deadline_date',
    #reschedule date
    #whatsapp conf
    'status',
    'completion_date',
    'review',
    'feedback'
    ]

In [20]:
reqdf.columns

Index(['task_id', 'task_description', 'status', 'priority', 'assigned_to',
       'created_by', 'completion_date', 'created_on', 'deadline_date',
       'category', 'review', 'feedback', 'BUId', 'BUType', 'ReportingBUId',
       'buname', 'UserID', 'UserName', 'U_BUID', 'RoleId', 'RoleName',
       'BRO_BUID', 'BRO_UserID', 'BRO_UserName', 'BM_BUID', 'BranchName',
       'BM_UserID', 'BM_UserName', 'BM_ReportingBUId', 'Hub_BUID',
       'Hub_UserID', 'Hub_UserName', 'HubName', 'Hub_ReportingBUId',
       'Region_BUID', 'Region_UserID', 'Region_UserName', 'RegionName',
       'Region_ReportingBUId', 'Div_BUID', 'Div_UserID', 'Div_UserName',
       'DivName'],
      dtype='object')

In [39]:
reqdf[reqdf['Region_UserID'] == 18035]

,task_id,task_description,status,priority,assigned_to,created_by,completion_date,created_on,deadline_date,category,review,feedback,BUId,BUType,ReportingBUId,buname,UserID,UserName,U_BUID,RoleId,RoleName,BRO_BUID,BRO_UserID,BRO_UserName,BM_BUID,BranchName,BM_UserID,BM_UserName,BM_ReportingBUId,Hub_BUID,Hub_UserID,Hub_UserName,HubName,Hub_ReportingBUId,Region_BUID,Region_UserID,Region_UserName,RegionName,Region_ReportingBUId,Div_BUID,Div_UserID,Div_UserName,DivName
116,129232,Customer (Customer id: Bgs:Mjl:01:089:011) wil...,Pending,1,30435,27518,None,2023-04-01,2023-04-01,3,None,None,393.0,5.0,607.0,Manjahaul,30435.0,Shiv Kumar Raut,393.0,55.0,BRO,393.0,30435.0,Shiv Kumar Raut,393.0,Manjahaul,13048.0,Rambabu Singh,607.0,607.0,3442.0,Shyam ji,Begusarai,1014.0,1014.0,18035.0,MANOJ KUMAR,Samastipur,653.0,653.0,10849.0,KESHRAJ YADAV,Muzaffarpur
139,129255,Customer (Customer id: Bgs:Bgu:01:031:025) has...,Pending,1,28154,27518,None,2023-04-01,2023-04-01,10,None,None,485.0,5.0,607.0,Begusarai,28154.0,Shrwan Kumar,485.0,55.0,BRO,485.0,28154.0,Shrwan Kumar,485.0,Begusarai,5133.0,SAROJ KUMAR,607.0,607.0,3442.0,Shyam ji,Begusarai,1014.0,1014.0,18035.0,MANOJ KUMAR,Samastipur,653.0,653.0,10849.0,KESHRAJ YADAV,Muzaffarpur
147,129263,Customer (Customer id: Smt:Smt:01:165:004) wil...,Pending,1,30917,27518,None,2023-04-01,2023-04-01,3,None,None,308.0,5.0,608.0,Samastipur City,30917.0,MANISHA KUMARI,308.0,55.0,BRO,308.0,30917.0,MANISHA KUMARI,308.0,Samastipur City,22486.0,Pankaj Kumar Mahto,608.0,608.0,2419.0,Akhilesh Kumar Singh,SAMASTIPUR,1014.0,1014.0,18035.0,MANOJ KUMAR,Samastipur,653.0,653.0,10849.0,KESHRAJ YADAV,Muzaffarpur
158,129274,Customer (Customer id: Smt:Smt:01:165:020) wil...,Pending,1,30917,27518,None,2023-04-01,2023-04-01,3,None,None,308.0,5.0,608.0,Samastipur City,30917.0,MANISHA KUMARI,308.0,55.0,BRO,308.0,30917.0,MANISHA KUMARI,308.0,Samastipur City,22486.0,Pankaj Kumar Mahto,608.0,608.0,2419.0,Akhilesh Kumar Singh,SAMASTIPUR,1014.0,1014.0,18035.0,MANOJ KUMAR,Samastipur,653.0,653.0,10849.0,KESHRAJ YADAV,Muzaffarpur
165,129281,Customer (Customer id: Smt:Smt:02:163:009) wil...,Pending,1,29786,27518,None,2023-04-01,2023-04-01,3,None,None,316.0,5.0,2199.0,Rosera,29786.0,Rupesh Kumar,316.0,55.0,BRO,316.0,29786.0,Rupesh Kumar,316.0,Rosera,7642.0,RAVI KUMAR I,2199.0,2199.0,7283.0,Anuj Kumar Srivastava,Dalsingh Sarai,1014.0,1014.0,18035.0,MANOJ KUMAR,Samastipur,653.0,653.0,10849.0,KESHRAJ YADAV,Muzaffarpur
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28699,172619,Customer (Customer id: Sam:SAM:403:212:004) wi...,Pending,1,28438,27518,None,2023-04-13,2023-04-19,3,None,None,403.0,5.0,608.0,Tajpur,28438.0,Ravindra Kumar,403.0,55.0,BRO,403.0,28438.0,Ravindra Kumar,403.0,Tajpur,25611.0,Raj Kumar Ray,608.0,608.0,2419.0,Akhilesh Kumar Singh,SAMASTIPUR,1014.0,1014.0,18035.0,MANOJ KUMAR,Samastipur,653.0,653.0,10849.0,KESHRAJ YADAV,Muzaffarpur
28790,172710,Customer (Customer id: Sam:Beg:393:151:012) ha...,Pending,1,31896,27518,None,2023-04-13,2023-04-13,10,None,None,393.0,5.0,607.0,Manjahaul,31896.0,SUBHASH KUMAR TANTI,393.0,55.0,BRO,393.0,31896.0,SUBHASH KUMAR TANTI,393.0,Manjahaul,13048.0,Rambabu Singh,607.0,607.0,3442.0,Shyam ji,Begusarai,1014.0,1014.0,18035.0,MANOJ KUMAR,Samastipur,653.0,653.0,10849.0,KESHRAJ YADAV,Muzaffarpur
28794,172714,Customer (Customer id: Smt:Smt:03:131:009) wil...,Pending,1,31017,27518,None,2023-04-13,2023-04-14,3,None,None,318.0,5.0,2199.0,Dalsingh Sarai,31017.0,RAM BABU MANJHI,318.0,55.0,BRO,318.0,31017.0,RAM BABU MANJHI,318.0,Dalsingh Sarai,17959.0,MEGHNATH SAHANI,2199.0,2199.0,7283.0,Anuj Kumar Srivastava,Dalsingh Sarai,1014.0,1014.0,18035.0,MANOJ KUMAR,Samastipur,653.0,653.0,10849.0,KESHRAJ YADAV,Muzaffarpur
28804,172724,Customer (Customer id: Sam:Beg:394:292:004) wi...,Pending,1,27567,27518,None,2023-04-13,2023-04-24,3,None,None,394.0,5.0,607.0,Balia,27567.0,Sanjeev Kumar Ram,394.

In [38]:
from datetime import datetime, date

In [22]:
for row in reqdf.head().itertuples():
    print(row)

Pandas(Index=0, task_id=129116, task_description='Customer (Customer id: Sdl:Bhr:01:006) is interested in a new loan.Loan Amount asked for is 100000 .Salun  is the loan purpose.Loan Documents have not been submitted.There is no change in Co-Applicant.There is no change in Bank AccountThere is no change in Address', status='Pending', priority=1, assigned_to=29938, created_by=2271, completion_date=None, created_on=datetime.date(2023, 4, 1), deadline_date=datetime.date(2023, 4, 1), category='7', review=None, feedback=None, BUId=459.0, BUType=5.0, ReportingBUId=2141.0, buname='Burhar', UserID=29938.0, UserName='Virendra Singh gound', U_BUID=459.0, RoleId=55.0, RoleName='BRO', BRO_BUID=459.0, BRO_UserID=29938.0, BRO_UserName='Virendra Singh gound', BM_BUID=459.0, BranchName='Burhar', BM_UserID=3366.0, BM_UserName='rajesh Sahu', BM_ReportingBUId=2141.0, Hub_BUID=2141.0, Hub_UserID=18022.0, Hub_UserName='Deepak Jagati', HubName='Burhar', Hub_ReportingBUId=2183.0, Region_BUID=2183.0, Region_Us

In [23]:
context = {}
context['r'] = reqdf

In [24]:
context['r']

,task_id,task_description,status,priority,assigned_to,created_by,completion_date,created_on,deadline_date,category,review,feedback,BUId,BUType,ReportingBUId,buname,UserID,UserName,U_BUID,RoleId,RoleName,BRO_BUID,BRO_UserID,BRO_UserName,BM_BUID,BranchName,BM_UserID,BM_UserName,BM_ReportingBUId,Hub_BUID,Hub_UserID,Hub_UserName,HubName,Hub_ReportingBUId,Region_BUID,Region_UserID,Region_UserName,RegionName,Region_ReportingBUId,Div_BUID,Div_UserID,Div_UserName,DivName
0,129116,Customer (Customer id: Sdl:Bhr:01:006) is inte...,Pending,1,29938,2271,None,2023-04-01,2023-04-01,7,None,None,459.0,5.0,2141.0,Burhar,29938.0,Virendra Singh gound,459.0,55.0,BRO,459.0,29938.0,Virendra Singh gound,459.0,Burhar,3366.0,rajesh Sahu,2141.0,2141.0,18022.0,Deepak Jagati,Burhar,2183.0,2183.0,1622.0,Pradeep kachhi,Katni,652.0,652.0,15190.0,JITENDRA YADAV,Jabalpur
1,129117,Customer (Customer id: Rab:cit:01:236:037) has...,Pending,1,30482,27199,None,2023-04-01,2023-04-01,10,None,None,79.0,5.0,584.0,Raibareli,30482.0,DHARMENDRA BAHADUR,79.0,55.0,BRO,79.0,30482.0,DHARMENDRA BAHADUR,79.0,Raibareli,1884.0,Vinod Kumar,584.0,584.0,15051.0,DHARMENDRA PRATAP SINGH,Raibarely,1002.0,1002.0,0.0,REGION HEAD NOT POSTED,Lucknow,651.0,651.0,7535.0,RAKESH KUMAR CHAUDHARY,Lucknow Division
2,129118,Customer (Customer id: Bab:cit:01:189:043) has...,Pending,1,27245,27199,None,2023-04-01,2023-04-01,10,None,None,8.0,5.0,583.0,Barabanki city,27245.0,ROLI KUMARI,8.0,55.0,BRO,8.0,27245.0,ROLI KUMARI,8.0,Barabanki city,3682.0,BHUPENDRA SINGH,583.0,583.0,3245.0,Krishan Kumar Yadav,Barabanki,1002.0,1002.0,0.0,REGION HEAD NOT POSTED,Lucknow,651.0,651.0,7535.0,RAKESH KUMAR CHAUDHARY,Lucknow Division
3,129119,Customer (Customer id: 1397268) will pay an am...,Pending,1,27815,27815,None,2023-04-01,2023-04-01,3,None,None,424.0,5.0,2139.0,Guna,27815.0,AAKASH YADAV,424.0,55.0,BRO,424.0,27815.0,AAKASH YADAV,424.0,Guna,2702.0,Prahlad Ahirwar,2139.0,2139.0,259.0,Vikash Namdev,GUNA,1013.0,1013.0,16347.0,Susheel Kumar Yadav,Gwalior,2156.0,2156.0,10851.0,Vinod singh,Gwalior
4,129120,Customer (Customer id: Lko:mhn:01:116:004) has...,Pending,1,31063,27199,None,2023-04-01,2023-04-01,10,None,None,184.0,5.0,583.0,Mohanlalganj,31063.0,SATYAM SINGH,184.0,55.0,BRO,184.0,31063.0,SATYAM SINGH,184.0,Mohanlalganj,21.0,MOHD IRFAN,583.0,583.0,3245.0,Krishan Kumar Yadav,Barabanki,1002.0,1002.0,0.0,REGION HEAD NOT POSTED,Lucknow,651.0,651.0,7535.0,RAKESH KUMAR CHAUDHARY,Lucknow Division
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28845,172765,Customer (Customer id: Sam:Mad:501:228:008) wi...,Pending,1,27745,3588,None,2023-04-13,2023-04-18,3,None,None,501.0,5.0,2165.0,Darbhanga,27745.0,Vishwanath Kumar,501.0,55.0,BRO,501.0,27745.0,Vishwanath Kumar,501.0,Darbhanga,3444.0,Shiv Prasad,2165.0,2165.0,0.0,HUB HEAD NOT POSTED,Darbhanga,2198.0,2198.0,10848.0,SHYAM KISHOR MISHRA,Darbhanga,653.0,653.0,10849.0,KESHRAJ YADAV,Muzaffarpur
28846,172766,Customer (Customer id: CHI:CHI:505:105:011) wi...,Pending,1,31254,29968,None,2023-04-13,2023-04-19,3,None,None,505.0,5.0,2134.0,Betul,31254.0,PALAK WAMANKAR,505.0,55.0,BRO,505.0,31254.0,PALAK WAMANKAR,505.0,Betul,5308.0,Roshani Warkade,2134.0,2134.0,17690.0,Sheel Dahiya,Betul,1023.0,1023.0,17692.0,ABHINAY KUMAR DUBEY,CHINDWADA,652.0,652.0,15190.0,JITENDRA YADAV,Jabalpur
28847,172767,Customer (Customer id: 2730224) will pay an am...,Pending,1,31893,31893,None,2023-04-13,2023-04-12,3,None,None,397.0,5.0,599.0,Kurukshetra,31893.0,ADARSH,397.0,55.0,BRO,397.0,31893.0,ADARSH,397.0,Kurukshetra,15139.0,AJAY SAHGAL,599.0,599.0,17668.0,Mohit Chauhan,Ladwa,2178.0,2178.0,4708.0,DIPENDRA CHAUHAN,Karnal,2157.0,2157.0,10846.0,SHASHI BHUSHAN SINGH,Saharanpur
28848,172768,Customer (Customer id: Var:MUG:448:166:018) wi...,Pending,1,31201,11905,None,2023-04-13,2023-04-15,3,None,None,448.0,5.0,2248.0,Chandauli City,31201.0,RITIK SINGH,448.0,55.0,BRO,448.0,31201.0,RITI

In [25]:
d = dict(zip(df['UserID'], df['UserName']))


In [26]:
reqdfcopy = reqdf.copy()

In [27]:
reqdfcopy['assigned_to'].apply(lambda x : str(x) + '-' + d[x] if x in d.keys() else str(x))

0        29938-Virendra Singh gound
1          30482-DHARMENDRA BAHADUR
2                 27245-ROLI KUMARI
3                27815-AAKASH YADAV
4               31063-SATYAM  SINGH
                    ...            
28845        27745-Vishwanath Kumar
28846         31254-PALAK  WAMANKAR
28847                  31893-ADARSH
28848             31201-RITIK SINGH
28849           28076-Amrendu Kumar
Name: assigned_to, Length: 28850, dtype: object

In [28]:
dlist=df[(df['RoleId']==34)&(df['RoleName']=='DIVISION HEAD')]['buname'].to_list()
df['userid_buid'] = df[df['RoleId']==34]['UserID'].astype(str) +"_"+ df[df['RoleId']==34]['U_BUID'].astype(str)
df2 = df.dropna(subset=['userid_buid'], how='all')
buid_list = df2['userid_buid'].to_list()
ddict= dict(zip(buid_list,dlist))
ddict


{'9739_650': 'Prayagraj Division',
 '7535_651': 'Lucknow Division',
 '15190_652': 'Jabalpur',
 '10849_653': 'Muzaffarpur',
 '7662_654': 'Moradabad',
 '17756_2154': 'Varanasi',
 '7575_2155': 'Jaipur',
 '10851_2156': 'Gwalior',
 '10846_2157': 'Saharanpur',
 '15111_2158': 'Patna',
 '22544_2200': 'Ranchi DO'}

In [29]:
from_date = datetime.date(datetime.strptime(str('2023-04-10'), '%Y-%m-%d'))
to_date = datetime.date(datetime.strptime(str('2023-04-12'), '%Y-%m-%d'))

In [30]:
reqdf[((reqdf['created_on'] >= from_date) & (reqdf['deadline_date'] <= to_date)) &( reqdf['DivName'] == 'Varanasi')]

,task_id,task_description,status,priority,assigned_to,created_by,completion_date,created_on,deadline_date,category,review,feedback,BUId,BUType,ReportingBUId,buname,UserID,UserName,U_BUID,RoleId,RoleName,BRO_BUID,BRO_UserID,BRO_UserName,BM_BUID,BranchName,BM_UserID,BM_UserName,BM_ReportingBUId,Hub_BUID,Hub_UserID,Hub_UserName,HubName,Hub_ReportingBUId,Region_BUID,Region_UserID,Region_UserName,RegionName,Region_ReportingBUId,Div_BUID,Div_UserID,Div_UserName,DivName
22699,166614,Customer (Customer id: Gor:Mah:2093:2748460) w...,Pending,1,27645,27168,None,2023-04-10,2023-04-11,3,None,None,2093.0,5.0,579.0,Pharenda,27645.0,Manish kumar,2093.0,55.0,BRO,2093.0,27645.0,Manish kumar,2093.0,Pharenda,11906.0,Shiv Poojan,579.0,579.0,4352.0,ramesh bahadur singh,Gorakhpur,1018.0,1018.0,0.0,REGION HEAD NOT POSTED,Gorakhpur,2154.0,2154.0,17756.0,Alok Singh,Varanasi
22760,166675,Customer (Customer id: NA) is interested in a ...,Pending,1,30367,11905,None,2023-04-10,2023-04-10,7,None,None,2101.0,5.0,2245.0,ROBERTSGANJ,30367.0,Vikas,2101.0,55.0,BRO,2101.0,30367.0,Vikas,2101.0,ROBERTSGANJ,18088.0,Ranjeet Kumar,2245.0,2245.0,24867.0,Anil Kumar Maurya,ROBERTSGANJ,1019.0,1019.0,0.0,REGION HEAD NOT POSTED,Varanasi,2154.0,2154.0,17756.0,Alok Singh,Varanasi
22796,166711,Customer (Customer id: Var:ADA:2088:053) is in...,Pending,1,31840,11905,None,2023-04-10,2023-04-10,7,None,None,2088.0,5.0,2142.0,Madhupur,31840.0,MD SHAHIL,2088.0,55.0,BRO,2088.0,31840.0,MD SHAHIL,2088.0,Madhupur,28105.0,Amit Kumar Singh,2142.0,2142.0,4059.0,PRABHAKAR PAL,ADALHAT,1019.0,1019.0,0.0,REGION HEAD NOT POSTED,Varanasi,2154.0,2154.0,17756.0,Alok Singh,Varanasi
22919,166835,Customer (Customer id: Gor:Gor:456:239:005) wi...,Pending,1,31386,27168,None,2023-04-10,2023-04-12,3,None,None,456.0,5.0,2295.0,Barhalganj,31386.0,VIPIN PASWAN,456.0,55.0,BRO,456.0,31386.0,VIPIN PASWAN,456.0,Barhalganj,4295.0,PRADEEP KUMAR RAJAK,2295.0,2295.0,1216.0,Naeem Ahmad,Kauriram,1018.0,1018.0,0.0,REGION HEAD NOT POSTED,Gorakhpur,2154.0,2154.0,17756.0,Alok Singh,Varanasi
23023,166939,Customer (Customer id: Var:Pan:444:264:019) wi...,Pending,1,30672,11905,None,2023-04-10,2023-04-12,3,None,None,444.0,5.0,557.0,Pandeypur,30672.0,SHALU MAURYA,444.0,55.0,BRO,444.0,30672.0,SHALU MAURYA,444.0,Pandeypur,4017.0,CHANDRA SHEKHER,557.0,557.0,10768.0,Devendra Pratap Singh,Varanasi,1019.0,1019.0,0.0,REGION HEAD NOT POSTED,Varanasi,2154.0,2154.0,17756.0,Alok Singh,Varanasi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27051,170967,Customer (Customer id: Gha:Bal:2153:070:005) h...,Pending,1,30981,2271,None,2023-04-12,2023-04-12,10,None,None,2153.0,5.0,2076.0,Bansdih,30981.0,MOHD AARIF,2153.0,55.0,BRO,2153.0,30981.0,MOHD AARIF,2153.0,Bansdih,25088.0,RAN PRATAP,2076.0,2076.0,1820.0,Gulab Yadav,Ballia,1025.0,1025.0,21479.0,Manoj Kumar,Ghazipur RO,2154.0,2154.0,17756.0,Alok Singh,Varanasi
27055,170971,Customer (Customer id: Gha:Gha:530:058:010) ha...,Pending,1,31115,2271,None,2023-04-12,2023-04-12,10,None,None,530.0,5.0,2144.0,Zamania,31115.0,AVINASH GUPTA,530.0,55.0,BRO,530.0,31115.0,AVINASH GUPTA,530.0,Zamania,4089.0,PINTU KUMAR GAUTAM,2144.0,2144.0,0.0,HUB HEAD NOT POSTED,NANDGANJ,1025.0,1025.0,21479.0,Manoj Kumar,Ghazipur RO,2154.0,2154.0,17756.0,Alok Singh,Varanasi
27270,171186,Customer (Customer id: Var:ADA:2089:052:023) w...,Pending,1,15165,11905,None,2023-04-12,2023-04-12,3,None,None,2089.0,5.0,2142.0,Adalhat,15165.0,Prabhakar Kumar Singh,2089.0,55.0,BRO,2089.0,15165.0,Prabhakar Kumar Singh,2089.0,Adalhat,4557.0,RAJ KUMAR,2142.0,2142.0,4059.0,PRABHAKAR PAL,ADALHAT,1019.0,1019.0,0.0,REGION HEAD NOT POSTED,Varanasi,2154.0,2154.0,17756.0,Alok Singh,Varanasi
27285,171201,Customer (Customer id: Var:ADA:2089:049:022) h...,Pending,1,26921,11905,None,2023-04-12,2023-04-12,10,None,None,2089.0,5.0,2142.0,Adalhat,26921.0,UMASHANKAR,2089.0,55.0,BRO,2089.0,26921.0,UMASHANKAR,2089.0,Adalhat,4557.0,R

In [31]:
reqdf[reqdf['assigned_to'] == 5133]

,task_id,task_description,status,priority,assigned_to,created_by,completion_date,created_on,deadline_date,category,review,feedback,BUId,BUType,ReportingBUId,buname,UserID,UserName,U_BUID,RoleId,RoleName,BRO_BUID,BRO_UserID,BRO_UserName,BM_BUID,BranchName,BM_UserID,BM_UserName,BM_ReportingBUId,Hub_BUID,Hub_UserID,Hub_UserName,HubName,Hub_ReportingBUId,Region_BUID,Region_UserID,Region_UserName,RegionName,Region_ReportingBUId,Div_BUID,Div_UserID,Div_UserName,DivName


In [47]:
reqdf.columns

Index(['task_id', 'task_description', 'status', 'priority', 'assigned_to',
       'created_by', 'completion_date', 'created_on', 'deadline_date',
       'category', 'review', 'feedback', 'BUId', 'BUType', 'ReportingBUId',
       'buname', 'UserID', 'UserName', 'U_BUID', 'RoleId', 'RoleName',
       'BRO_BUID', 'BRO_UserID', 'BRO_UserName', 'BM_BUID', 'BranchName',
       'BM_UserID', 'BM_UserName', 'BM_ReportingBUId', 'Hub_BUID',
       'Hub_UserID', 'Hub_UserName', 'HubName', 'Hub_ReportingBUId',
       'Region_BUID', 'Region_UserID', 'Region_UserName', 'RegionName',
       'Region_ReportingBUId', 'Div_BUID', 'Div_UserID', 'Div_UserName',
       'DivName'],
      dtype='object')

In [48]:
reqdf[reqdf['Hub_BUID'] == 18043]

,task_id,task_description,status,priority,assigned_to,created_by,completion_date,created_on,deadline_date,category,review,feedback,BUId,BUType,ReportingBUId,buname,UserID,UserName,U_BUID,RoleId,RoleName,BRO_BUID,BRO_UserID,BRO_UserName,BM_BUID,BranchName,BM_UserID,BM_UserName,BM_ReportingBUId,Hub_BUID,Hub_UserID,Hub_UserName,HubName,Hub_ReportingBUId,Region_BUID,Region_UserID,Region_UserName,RegionName,Region_ReportingBUId,Div_BUID,Div_UserID,Div_UserName,DivName
